## Objects
Objects are abstract representations of everything on an operating system. An object can be as simple as an array of numbers or as complicated as the thread of a process. The best way to think about an object is that is the data structure that contains all the information regarding a particular 'thing'. An array object would contain information about the numbers it held, how many number it held, what order they were in etc. A thread object would contain far more information: the thread count, the owning process, the memory in use, the executable image in memory, the files it had access to, its permissions on the system etc. On Windows, practically everything is an object.

Objects are made up of three components: the objects _type_, its _methods_ and its _properties_.

## Types
PowerShell is built with .NET, meaning all its objects are ultimately built using .NET types. Types describe what features an object possess and the kinds of actions that it can take or be taken on it. In .NET all types are ultimately derived from the System.Object base type. 

In [1]:
# note: parentheses in PS are referred to as the 'grouping' operator, and are used around this cmdlet to capture the cmdlets output, so that output can 
# then be used in the below expression alongside the string "Cmdlet Output"
"Cmdlet Output: " + (Get-Date);
"Type: " + (Get-Date).GetType().FullName;
"Base Type: " + (Get-Date).GetType().BaseType;

Cmdlet Output: 05/01/2024 17:54:26
Type: System.DateTime
Base Type: System.ValueType


Objects can be _cast_ between different types using the square brackets and the type to cast to.

In [2]:
$date = Get-Date
"Type: " + $date.GetType().FullName;
"New Type: " + ([String]$date).GetType().FullName;

Type: System.DateTime
New Type: System.String


Variables can be constrained to a certain type using the same casting syntax.

In [3]:
[System.DateTime]$i = "Hello"
[System.DateTime]$j = "02/03/22"
Write-Output "j contains: $j"

MetadataError: Cannot convert value "Hello" to type "System.DateTime". Error: "The string 'Hello' was not recognized as a valid DateTime. There is an unknown word starting at index '0'."
j contains: 02/03/2022 00:00:00


Error: Command failed: SubmitCode: [System.DateTime]$i = "Hello" ...

PowerShell objects can be created for newly any .NET type. Creating an object of a specific type usually happens through casting as shown above, and when this happens PowerShell will automatically select the most appropriate _constructor_ for that type. The constructor can also be called explicitly however, using the `new` static method that is common to all .NET types.

In [1]:
$google = "http://www.google.com"
$google.GetType()
$google = [System.uri]::new("http://www.google.com")
$google.GetType()


IsPublic IsSerial Name                                     BaseType
-------- -------- ----                                     --------
True     True     String                                   System.Object
True     True     Uri                                      System.Object



## Methods
Just like in OOP languages like C++, Java or C#, PowerShell objects have methods which describe the kinds of operations that can be performed on the object.

In [4]:
$strObj = "This is a string object" 
$strObj | Get-Member -MemberType Methods


   TypeName: System.String

Name                 MemberType Definition
----                 ---------- ----------
Clone                Method     System.Object Clone(), System.Object ICloneable.Clone()
CompareTo            Method     int CompareTo(System.Object value), int CompareTo(string strB), in…
Contains             Method     bool Contains(string value), bool Contains(string value, System.St…
CopyTo               Method     void CopyTo(int sourceIndex, char[] destination, int destinationIn…
EndsWith             Method     bool EndsWith(string value), bool EndsWith(string value, System.St…
EnumerateRunes       Method     System.Text.StringRuneEnumerator EnumerateRunes()
Equals               Method     bool Equals(System.Object obj), bool Equals(string value), bool Eq…
GetEnumerator        Method     System.CharEnumerator GetEnumerator(), System.Collections.IEnumera…
GetHashCode          Method     int GetHashCode(), int GetHashCode(System.StringComparison compari…
GetPinnableRefe

Methods are accessed with dot . notation, and the output of a method is itself an object with its own set of methods.

In [10]:
"Output of .EndsWith() method: " + $strObj.EndsWith('object')
"Type of output of .EndsWith() method: " + ($strObj.EndsWith('object').GetType().FullName)

Output of .EndsWith() method: True
Type of output of .EndsWith() method: System.Boolean


As can be seen above, methods can be called on objects that are generated by commands, either by storing the object in a variable or using the ( ) grouping operator.

In [3]:
$date = Get-Date;
"Method called on object stored in a variable: " + $date.DateTime;
"Method called on object emitted from Get-Date command: " + (Get-Date).DateTime;

Method called on object stored in a variable: Monday, March 11, 2024 11:00:35 AM
Method called on object emitted from Get-Date command: Monday, March 11, 2024 11:00:36 AM


## Properties
While methods are the actions that can performed on an object and the data the object represents, properties describe the data itself.

In [6]:
$strObj | Get-Member -MemberType Property
"strObj is " + $strObj.Length + " characters long"


   TypeName: System.String

Name   MemberType Definition
----   ---------- ----------
Length Property   int Length {get;}
strObj is 23 characters long



## Custom Objects
It is possible to create objects of a custom type by using a _hashtable_ data structure and casting it to the type **\[PSCustomObject\]**

In [7]:
# the hash table syntax of []@ is explained in later notebooks!
$Foo = [pscustomobject]@{ Bar="YELLOWSUBMARINE" };
$Foo | Get-Member -MemberType Properties


   TypeName: System.Management.Automation.PSCustomObject

Name MemberType   Definition
---- ----------   ----------
Bar  NoteProperty string Bar=YELLOWSUBMARINE



## Selecting Parts of Objects
The `Select-Object` cmdlet creates a new custom object from the selected properties of the object that it receives via the pipeline. Technically it return the input object but with only the selected set of properties.

In [12]:
Get-Process code | Select-Object -First 1 -Property Path


Path
----
C:\Users\alex\AppData\Local\Programs\Microsoft VS Code\Code.exe



Examining the object shows the it contains only the properties selected from the previous object, and the type is a custom type based on the type of the input object.

In [9]:
"===> New custom object with only select properties <===" 
(Get-Process code | Select-Object -First 1 -Property Path) | Get-Member

"`n===> Selected.System.Diagnostics.Process is itself a kind of custom powershell object <==="
(Get-Process code | Select-Object -First 1 -Property Path).GetType().FullName

"`n===> Put another way, Selected.System.Diagnostics.Process is constructed using the PSCustomObject type <==="
(Get-Process code | Select-Object -First 1 -Property Path).pstypenames.ForEach{ return $_ + " which inherits from..."} 


===> New custom object with only select properties <===

   TypeName: Selected.System.Diagnostics.Process

Name        MemberType   Definition
----        ----------   ----------
Equals      Method       bool Equals(System.Object obj)
GetHashCode Method       int GetHashCode()
GetType     Method       type GetType()
ToString    Method       string ToString()
Path        NoteProperty System.String Path=C:\Users\alex\AppData\Local\Programs\Microsoft VS Code…

===> Selected.System.Diagnostics.Process is itself a kind of custom powershell object <===
System.Management.Automation.PSCustomObject

===> Put another way, Selected.System.Diagnostics.Process is constructed using the PSCustomObject type <===
Selected.System.Diagnostics.Process which inherits from...
System.Management.Automation.PSCustomObject which inherits from...
System.Object which inherits from...

